In [1]:
import os, sys
import pandas as pd
from pydub import AudioSegment

from preprocessing_code.Preprocessing_Cutting_Merging import (
    get_wav_time, get_second_part_wav, 
)

def wav_combine(path_wav_merge_list, path_wav_out):
    sounds = [AudioSegment.from_wav(x) for x in path_wav_merge_list]
    playlist = AudioSegment.empty()
    for sound in sounds: playlist += sound
    playlist.export(path_wav_out, format="wav")


## segment form csv or excel 

In [17]:
# The cut audio directory, which is also the audio directory to be merged.
path_segment = os.path.join(os.getcwd(), 'preprocessing_code', 'data_segment')
path_dir_excel = os.path.join(os.getcwd(), 'preprocessing_code', 'data_time')
path_dir_audio = os.path.join(os.getcwd(), '..', 'daic_woz', 'audio')
if not os.path.exists(path_segment): os.makedirs(path_segment)

a_l = []  # Record the number of audio cuts
print('Start cutting the audio!')

for root, dir, files in os.walk(path_dir_excel):
    files = sorted(files)
    for file in files:
        if file.endswith('.xlsx') and file[0].isdigit() : # * check daic_woz excel 
            user_id_str = file.split('_TRANSCRIPT')[0]
            audio_file = f'{user_id_str}_AUDIO.wav'
            path_audio = os.path.join(path_dir_audio, audio_file)
            path_excel = os.path.join(path_dir_excel, file)
            time_all = get_wav_time(path_audio)

            print(f"The audio  is {audio_file}, the file is {file}")
            df = pd.read_excel(path_excel, usecols=['start_time', 'stop_time'])

            index = 1  # Cut serial number names, starting with serial number 1
            k = 0
            # k_l = 0  # Total number of audio cuts recorded
            for j in df:
                l = len(df)
                while k <= l - 1:
                    dict_data = df.iloc[k].to_dict()
                    start_time = float(dict_data['start_time'])
                    stop_time = float(dict_data['stop_time'])

                    audio_segment = os.path.join(path_segment, f"{audio_file[:-len('.wav')]}_{index}.wav" )  
                    get_second_part_wav(path_audio, start_time, stop_time, audio_segment)

                    index += 1
                    k += 1
        a_l.append(k)
        # print(a_l)
        print("Cutting out %d audio" % l)

print('A total of %d audio cuts' % sum(a_l))
print(a_l)
print('Audio cutting complete!')


Start cutting the audio!
Cutting out 39 audio
The audio  is 300_AUDIO.wav, the file is 300_TRANSCRIPT.xlsx
Cutting out 39 audio
The audio  is 301_AUDIO.wav, the file is 301_TRANSCRIPT.xlsx
Cutting out 104 audio
The audio  is 302_AUDIO.wav, the file is 302_TRANSCRIPT.xlsx
Cutting out 97 audio
The audio  is 303_AUDIO.wav, the file is 303_TRANSCRIPT.xlsx
Cutting out 103 audio
The audio  is 304_AUDIO.wav, the file is 304_TRANSCRIPT.xlsx
Cutting out 104 audio
The audio  is 305_AUDIO.wav, the file is 305_TRANSCRIPT.xlsx
Cutting out 288 audio
The audio  is 306_AUDIO.wav, the file is 306_TRANSCRIPT.xlsx
Cutting out 108 audio
The audio  is 307_AUDIO.wav, the file is 307_TRANSCRIPT.xlsx
Cutting out 186 audio
The audio  is 308_AUDIO.wav, the file is 308_TRANSCRIPT.xlsx
Cutting out 154 audio
The audio  is 309_AUDIO.wav, the file is 309_TRANSCRIPT.xlsx
Cutting out 94 audio
The audio  is 310_AUDIO.wav, the file is 310_TRANSCRIPT.xlsx
Cutting out 148 audio
The audio  is 311_AUDIO.wav, the file is 311

/Users/pataweeprakrankamanant/anaconda3/envs/maywe3.10/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Cutting out 144 audio
The audio  is 379_AUDIO.wav, the file is 379_TRANSCRIPT.xlsx
Cutting out 225 audio
The audio  is 380_AUDIO.wav, the file is 380_TRANSCRIPT.xlsx
Cutting out 368 audio
The audio  is 381_AUDIO.wav, the file is 381_TRANSCRIPT.xlsx
Cutting out 188 audio
The audio  is 382_AUDIO.wav, the file is 382_TRANSCRIPT.xlsx
Cutting out 157 audio
The audio  is 383_AUDIO.wav, the file is 383_TRANSCRIPT.xlsx
Cutting out 224 audio
The audio  is 384_AUDIO.wav, the file is 384_TRANSCRIPT.xlsx
Cutting out 156 audio
The audio  is 385_AUDIO.wav, the file is 385_TRANSCRIPT.xlsx
Cutting out 55 audio
The audio  is 386_AUDIO.wav, the file is 386_TRANSCRIPT.xlsx
Cutting out 186 audio
The audio  is 387_AUDIO.wav, the file is 387_TRANSCRIPT.xlsx
Cutting out 89 audio
The audio  is 388_AUDIO.wav, the file is 388_TRANSCRIPT.xlsx
Cutting out 92 audio
The audio  is 389_AUDIO.wav, the file is 389_TRANSCRIPT.xlsx
Cutting out 126 audio
The audio  is 390_AUDIO.wav, the file is 390_TRANSCRIPT.xlsx
Cutting

## sorted file_path 

In [22]:
path_wav_list = [x for x in os.listdir(path_segment) if x.endswith('.wav') and x[0].isdigit()]
# * example file wav name : 300_AUDIO_14.wav 
path_wav_list = sorted(path_wav_list, key=lambda x: 
                    (int(x.split('_')[0]), int(x.split('_')[-1].split('.')[0])))
path_wav_list = [os.path.join(path_segment, x) for x in path_wav_list]
# path_wav_list

In [25]:
# Initial value setting

print('Start merging audio!')
num_merge = 5 
err_id_list = [341, 393, 397, 459]

path_dir_audio_combine = os.path.join(os.getcwd(), 'preprocessing_code','data_audio_combine')
if not os.path.exists(path_dir_audio_combine): os.makedirs(path_dir_audio_combine)

filename_list = [os.path.basename(path) for path in path_wav_list]
user_id_list = [os.path.basename(path) for path in filename_list]
user_id_list = sorted(set([int(x.split('_')[0]) for x in user_id_list]))

# user_id_list
for user_id in user_id_list:
    if not user_id in err_id_list:
        filename_user_list = [x for x in filename_list if x.startswith(str(user_id))]
        filename_user_list = sorted(filename_user_list, 
                                    key=lambda x: int(x.split('_')[-1].split('.')[0]))
        path_sel_list = [os.path.join(path_segment, x) for x in filename_user_list]
        for idx, file_merge_list in enumerate([path_sel_list[i:i + num_merge] for i in range(0, len(path_sel_list), num_merge)]):
            out_path = os.path.join(path_dir_audio_combine, f"{user_id}_{idx+1}.wav")
            wav_combine(path_wav_merge_list = file_merge_list, 
                        path_wav_out = out_path)
print('Audio merge complete!')


[300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 395,
 396,
 397,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470

## merging audio

In [ ]:
print('*' * 50)
print('Start merging audio!')

err_id_list = [341, 393, 397, 459]

path_dir_audio_combine = os.path.join(os.getcwd(), 'preprocessing_code','data_audio_combine')
if not os.path.exists(path_dir_audio_combine): os.makedirs(path_dir_audio_combine)

# Initial value setting
n = 5 



## understand function preprocessing 

In [5]:

path_wav_test = os.path.join(os.getcwd(), '..', 'daic_woz', 'audio', '300_AUDIO.wav')
# sorted(os.listdir(path_wav_test))
print(f'path_wav_test : {path_wav_test}')
print(f'get_wav_time(path_wav_test) : {get_wav_time(path_wav_test)}')


path_wav_test : /Users/pataweeprakrankamanant/Desktop/Depression_Recognition/../daic_woz/audio/300_AUDIO.wav
get_wav_time(path_wav_test) : 648.5


In [13]:
path_excel_test = '/Users/pataweeprakrankamanant/Desktop/Depression_Recognition/preprocessing_code/data_time/300_TRANSCRIPT.xlsx'

df = pd.read_excel(path_excel_test, usecols=['start_time', 'stop_time'])

In [14]:
df

,start_time,stop_time
0,62.328,63.178
1,68.978,70.288
2,75.028,78.128
3,83.808,84.588
4,88.458,89.968
5,90.278,93.568
6,94.738,95.298
7,96.588,97.278
8,102.428,103.268
9,104.278,105.558
